In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard
import time
import matplotlib.pyplot as plt
from matplotlib import gridspec

In [2]:
NAME = "cifakeCNN{}".format(time.strftime("%Y%m%d-%H%M%S"))

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [4]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
#   try:
#     tf.config.set_logical_device_configuration(
#         gpus[0],
#         [tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Virtual devices must be set before GPUs have been initialized
#     print(e)

In [5]:
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)

In [6]:
ds_train = image_dataset_from_directory(
    '../cifar-10-stable-diffusion-detection/train',
    labels='inferred',
    label_mode='binary',
    image_size=[32,32],
    interpolation='nearest',
    batch_size=32,
    shuffle=True,
    seed=69,
    #validation_split=None,
    subset=None,
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 100000 files belonging to 2 classes.


In [7]:
ds_test = image_dataset_from_directory(
    '../cifar-10-stable-diffusion-detection/test',
    labels='inferred',
    label_mode='binary',
    image_size=[32,32],
    interpolation='nearest',
    batch_size=32,
    shuffle=True,
    seed=69,
    #validation_split=None,
    subset=None,
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 20000 files belonging to 2 classes.


In [8]:
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE


In [9]:
ds_train = (
    ds_train
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)


In [10]:
ds_valid = (
    ds_test
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

In [11]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=8,
    min_delta=0.001,
    restore_best_weights=True)

In [12]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=0.0008)

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint
timestamp = time.strftime("%Y%m%d-%H%M%S")
model_checkpoint = ModelCheckpoint(
    f'best/best_model_{timestamp}.keras', monitor='val_loss', save_best_only=True, mode='min'
)

In [14]:
from tensorflow.keras.activations import swish
def MBConv1(input_tensor, hp=None, expansion=4, out_channels=16, strides=(1,   1), use_se=True):
    # Expansion
    x = layers.Conv2D(expansion, (1,   1), padding='same', use_bias=False)(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)

    # Depthwise 
    x = layers.DepthwiseConv2D(kernel_size=(3,   3), strides=strides, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x) 

    # Pointwise 
    x = layers.Conv2D(out_channels, (1,   1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # Squeeze-and-Excitation
    if use_se:
        se_shape = (1,  1, out_channels)
        se = layers.GlobalAveragePooling2D()(x)
        se = layers.Reshape((1,  1, out_channels))(se) 
        se = layers.Conv2D(out_channels //  4, (1,  1), padding='same', use_bias=True)(se)  # Use a kernel size of (1,  1)
        se = layers.Activation(swish)(se)   
        se = layers.Conv2D(out_channels, (1,  1), padding='same', use_bias=True)(se)  # Use a kernel size of (1,  1)
        se = layers.Activation('sigmoid')(se)
        se = layers.Reshape(se_shape[1:])(se)  # Reshape to original
        x = layers.Multiply()([x, se])

    # Residual
    if input_tensor.shape[-1] == out_channels and strides == (1,   1):
        x = layers.Add()([x, input_tensor])
    else:
        input_tensor = layers.Conv2D(out_channels, (1,   1), strides=strides, padding='same', use_bias=False)(input_tensor)
        x = layers.Add()([x, input_tensor])

    return x

input_tensor = layers.Input(shape=(32,  32,  3)) 
output_tensor = MBConv1(input_tensor)
model = keras.models.Model(inputs=input_tensor, outputs=output_tensor)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 4)    12          ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 4)   16          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 4)    0           ['batch_normalization[0][0]']

In [25]:

def MBConv6(input_tensor, hp=None, out_channels=32, expansion=6, use_se=True, dropout_rate=0.0):
    # Inverted Residual Structure
    # Pointwise Convolution  1 with Expansion Factor  6
    x = layers.Conv2D(expansion * input_tensor.shape[-1], (1,  1), padding='same', use_bias=False)(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)

    # Depthwise Separable Convolution with stride (1,  1)
    x = layers.DepthwiseConv2D(kernel_size=(3,  3), strides=(1,  1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(swish)(x)

    # Pointwise Convolution  2
    x = layers.Conv2D(out_channels, (1,  1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # Dropout
    if dropout_rate >  0.0:
        x = layers.Dropout(dropout_rate)(x)

    # Bottleneck Convolution  
    x = layers.Conv2D(out_channels, (1,  1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # Squeeze-and-Excitation
    if use_se:
        se_shape = (1,  1, out_channels)
        se = layers.GlobalAveragePooling2D()(x)
        se = layers.Reshape(se_shape)(se)
        se = layers.Conv2D(out_channels //  4, (1,  1), padding='same', use_bias=True)(se)
        se = layers.Activation(swish)(se)
        se = layers.Conv2D(out_channels, (1,  1), padding='same', use_bias=True)(se)
        se = layers.Activation('sigmoid')(se)
        x = layers.Multiply()([x, se])

    # Residual
    if input_tensor.shape[-1] == out_channels:
        shortcut = input_tensor
    else:
        shortcut = layers.Conv2D(out_channels, (1,  1), strides=(1,  1), padding='same', use_bias=False)(input_tensor)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])

    return x
input_tensor = layers.Input(shape=(32,  32,  16)) 
output_tensor = MBConv6(input_tensor)
model = keras.models.Model(inputs=input_tensor, outputs=output_tensor)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 32, 32, 16)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d_27 (Conv2D)             (None, 32, 32, 96)   1536        ['input_9[0][0]']                
                                                                                                  
 batch_normalization_22 (BatchN  (None, 32, 32, 96)  384         ['conv2d_27[0][0]']              
 ormalization)                                                                                    
                                                                                            

In [26]:
import keras_tuner
from keras_tuner import RandomSearch
from tensorflow.keras.models import Model

def build_model(hp=None):
    activation_function = 'leaky_relu' if hp.Boolean(f'use_leaky_relu_conv') else 'relu'
    input_tensor = layers.Input(shape=[32,  32,  3])
    
    x = layers.experimental.preprocessing.RandomRotation(factor=0.1488228133769420)(input_tensor)
    x = layers.experimental.preprocessing.RandomTranslation(height_factor=0.069, width_factor=0.069)(x)
    x = layers.experimental.preprocessing.RandomFlip(mode='horizontal')(x)
    x = layers.BatchNormalization()(x)
    

    x = MBConv1(x, hp=hp)
    x = MBConv6(input_tensor, hp=hp, out_channels=hp.Int('out_channels', min_value=28, max_value=36, step=4),
                expansion=hp.Int('expansion', min_value=4, max_value=8, step=2),
                use_se=hp.Boolean('use_se'),
                dropout_rate=hp.Float('conv6_dropout_rate', min_value=0.0, max_value=0.2, step=0.1))
    x = MBConv6(input_tensor, hp=hp, out_channels=hp.Int('out_channels', min_value=28, max_value=36, step=4),
                expansion=hp.Int('expansion', min_value=4, max_value=8, step=2),
                use_se=hp.Boolean('use_se'),
                dropout_rate=hp.Float('conv6_dropout_rate', min_value=0.0, max_value=0.2, step=0.1))
    x = MBConv6(input_tensor, hp=hp, out_channels=hp.Int('out_channels', min_value=28, max_value=36, step=4),
                expansion=hp.Int('expansion', min_value=4, max_value=8, step=2),
                use_se=hp.Boolean('use_se'),
                dropout_rate=hp.Float('conv6_dropout_rate', min_value=0.0, max_value=0.2, step=0.1))
    if hp.Boolean('use_maxpool'):
        x = layers.MaxPool2D()(x)

    x = layers.Conv2D(filters=hp.Int('conv_filter', min_value=2, max_value=4, step=2),
                      kernel_size=hp.Choice('conv_kernel', values=[3,  7]),
                      activation=activation_function, padding='same',
                      kernel_regularizer=layers.regularizers.l2(0.002) if hp.Boolean('l2_reg') else None)(x)
    x = layers.BatchNormalization()(x)
    if hp.Boolean('use_maxpool'):
        x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters=hp.Int('conv_filter', min_value=2, max_value=4, step=2),
                      kernel_size=hp.Choice('conv_kernel', values=[3,  7]),
                      activation=activation_function, padding='same',
                      kernel_regularizer=layers.regularizers.l2(0.002) if hp.Boolean('l2_reg') else None)(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Flatten()(x)
    
    activation_function = 'leaky_relu' if hp.Boolean(f'use_leaky_relu_dense') else 'relu'
    for i in range(hp.Int('num_dense_layers', min_value=0, max_value=4, step=2)):   
        x = layers.Dense(hp.Int(f'dense_{i+1}_units', min_value=1, max_value=513, step=256),
                         activation=activation_function,
                         kernel_regularizer=layers.regularizers.l2(0.002) if hp.Boolean('l2_reg') else None)(x)
        x = layers.Dropout(hp.Float(f'dropout_{i+1}_rate', min_value=0.0, max_value=0.5, step=0.1))(x)
    
    x = layers.Dense(1, activation='sigmoid')(x)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=0.001, max_value=0.1, sampling='LOG'),
                                         epsilon=0.01)
    model = Model(inputs=input_tensor, outputs=x)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['binary_accuracy'])
    return model


In [27]:
tuner = RandomSearch(
    build_model,
    objective='val_binary_accuracy',
    max_trials=42,
    executions_per_trial=3,
    directory='projects',
    project_name='cifake4')

In [ ]:
# Start the hyperparameter search with early stopping
tuner.search(ds_train, epochs=50, validation_data=ds_valid, callbacks=[early_stopping, tensorboard, reduce_lr, model_checkpoint])

best_model = tuner.get_best_models(num_models=1)[0]


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
True              |True              |use_leaky_relu_conv
32                |32                |out_channels
4                 |4                 |expansion
False             |False             |use_se
0.2               |0.2               |conv6_dropout_rate
False             |False             |use_maxpool
4                 |4                 |conv_filter
3                 |3                 |conv_kernel
False             |False             |l2_reg
False             |False             |use_leaky_relu_dense
2                 |2                 |num_dense_layers
0.030596          |0.030596          |learning_rate

Epoch 1/50
 101/3125 [..............................] - ETA: 5:21 - loss: 0.6329 - binary_accuracy: 0.6593

In [ ]:
best_model.summary()

In [ ]:
tuner.results_summary()

In [ ]:
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
hyperparameters_str = "_".join(f"{k}={v}__" for k, v in best_hyperparameters.get_config().items())
NAME += "_" + hyperparameters_str

best_model.save(f"best/{NAME}.keras")